In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## Data Preprocess

In [2]:
%%time
data = pd.read_csv("data.csv", low_memory=False)
data.columns = ["Code", "Company", "Industry", "Date", "Open", "High", "Low", "Close", "Volume", "MV", "PB"]
data.Date = data.Date.apply(lambda x : datetime.strptime(x, "%Y/%m/%d"))
data.head()

Wall time: 50.2 s


**MV** is Market Value(in million NTD), **PB** is Price to Book ratio.

In [3]:
%%time
#find which years included in data
data_year = data.Date.apply(lambda x : x.year).unique()

#find unique trading date in data
data_date = pd.DataFrame(data.Date.unique(), columns = ["Date"])

#find first trading date for each year, these dates will be used as entry points/exit points during backtesting
first_date_each_year = []
for year in data_year:
    temp = data_date[data_date.Date.apply(lambda x : x.year == year)].min()["Date"]
    first_date_each_year.append(temp)

Wall time: 13.3 s


## Backtest

In [4]:
def selected_company(data = data, num_selected = 10, by = "MV", ascending = True, least_volume = 10, trade_mode = "A"):
    
    data = data
    #empty dataframe to collect result
    selected_data = pd.DataFrame([])
    for i in range(len(first_date_each_year)):
        
        #entry point
        date_in = first_date_each_year[i]
        #exit point == a year after entry point (use the latest date of data when dealing with last year)
        try:
            date_out = first_date_each_year[i + 1]
        except:
            date_out = data_date.iloc[-1, :]["Date"]
            
        #deal with entry trade
        #selected companies need to possess trading data on appointed date
        temp_data_in = data[data.Volume >= least_volume]
        #also need to meet the requirement of least trading volume(in case of liquidity risk)
        temp_data_in = temp_data_in[temp_data_in.Date == date_in]
        temp_data_in = temp_data_in.sort_values(by = by, ascending = ascending).reset_index(drop = True)
        temp_data_in = temp_data_in.iloc[0:num_selected, :]
        
        #deal with exit trade
        temp_data_out = pd.DataFrame([])
        for code in temp_data_in.Code:
            temp_company = data[data.Code == code]
            temp_date_out = pd.DataFrame(temp_company.Date - date_out).Date.apply(lambda x : x.days)
            try:
                #find the closest exit point according to appointed point
                temp_date_out = temp_company.Date[temp_date_out[temp_date_out >= 0].idxmin()]
                temp_result_out = temp_company[temp_company.Date == temp_date_out]
                temp_result_out.columns = (data.columns + "_out")
                
                #if exit point's trade volume don't meet the requirement of least volume, then do:
                if temp_result_out["Volume_out"].values < least_volume:
                    #in trade mode "A, set exit price = 0(more realistic, because we maybe unable to sell at such a low liquidity)
                    if trade_mode == "A":
                        temp_result_out = pd.DataFrame([[0] * len(data.columns)], columns = (data.columns + "_out"))
                    #in trade mode "B", set exit price = entry price
                    if trade_mode == "B":
                        temp_result_out = temp_data_in[temp_data_in.Code == code]
                        temp_result_out.columns = (data.columns + "_out")
                    
            #if the company was unlisted, then for loop would jump to except, set exit price = 0
            except:
                temp_result_out = pd.DataFrame([[0] * len(data.columns)], columns = (data.columns + "_out"))

            temp_data_out = pd.concat([temp_data_out, temp_result_out], axis = 0)
            temp_data_out.reset_index(inplace = True, drop = True)
        
        temp_selected_data = pd.concat([temp_data_in,  temp_data_out], axis = 1)
        selected_data = pd.concat([selected_data, temp_selected_data], axis = 0)
        
    total_return = (sum(selected_data["Close_out"]) - sum(selected_data["Close"])) / sum(selected_data["Close"])
    return selected_data# total_return

In [5]:
def calculate_return(data, weight_mode = "equal", initial_capital = 10000):
    
    for date in data.Date.unique():
        temp = data[data.Date == date]
        
        if weight_mode == "equal":
            weight_array = np.array([1/len(temp)] * len(temp))
        if weight_mode == "MV_based":
            weight_array = np.array(temp.MV / temp.MV.sum())
            
        temp_return =  ((temp.Close_out - temp.Close) / temp.Close + 1)
        capital_allocation = initial_capital * weight_array
        initial_capital = np.dot(capital_allocation, temp_return)
        print(capital_allocation, initial_capital)
        
    return initial_capital

In [6]:
%%time
test = selected_company(trade_mode='A')
calculate_return(test)

[ 1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.] 1696.46703523
[ 169.64670352  169.64670352  169.64670352  169.64670352  169.64670352
  169.64670352  169.64670352  169.64670352  169.64670352  169.64670352] 3212.73475426
[ 321.27347543  321.27347543  321.27347543  321.27347543  321.27347543
  321.27347543  321.27347543  321.27347543  321.27347543  321.27347543] 3321.70480582
[ 332.17048058  332.17048058  332.17048058  332.17048058  332.17048058
  332.17048058  332.17048058  332.17048058  332.17048058  332.17048058] 513.73600247
[ 51.37360025  51.37360025  51.37360025  51.37360025  51.37360025
  51.37360025  51.37360025  51.37360025  51.37360025  51.37360025] 456.93468084
[ 45.69346808  45.69346808  45.69346808  45.69346808  45.69346808
  45.69346808  45.69346808  45.69346808  45.69346808  45.69346808] 559.440645398
[ 55.94406454  55.94406454  55.94406454  55.94406454  55.94406454
  55.94406454  55.94406454  55.94406454  55.94406454  55.94406454] 1103.24051468
[ 11

In [7]:
%%time
test = selected_company(trade_mode='A')
calculate_return(test, weight_mode = "MV_based")

[  469.41678521   633.00142248   647.22617354   732.57467994   896.15931721
  1095.30583215  1244.66571835  1372.68847795  1450.92460882  1458.03698435] 2191.96430625
[ 114.11994915  178.71614678  180.86935336  228.23989829  232.54631147
  234.69951806  243.31234441  254.07837735  258.38479052  266.99761687] 4531.41205396
[ 179.79375332  255.9775471   356.54015488  380.91896889  393.1083759
  499.76568719  563.76007396  585.09153622  652.13327475  664.32268175] 6428.4802812
[ 390.39353934  427.98699128  615.95425097  650.65589891  650.65589891
  708.49197881  708.49197881  743.19362675  754.76084273  777.89527469] 1084.36975102
[  78.5233268    84.399222     99.35604615  100.42439073  106.83445823
  114.8470426   115.38121489  126.06466071  126.598833    131.94055591] 927.083147861
[  58.58440739   62.81214813   73.0795185    91.80237034   94.21822219
   97.84199997  111.12918515  111.73314811  112.33711108  113.545037  ] 1118.68491739
[  50.08026853   80.53448589   83.24152743   99.48